## THIRD EXPERIMENT

Для третьего эксперимента был использован следующий подход: у нас есть готовая SfM (которую мы получили из 2го эксперимента), при помощи которой мы хотим создать новую SfM с заменой 3д точек. 3D точки были взяты из GT-меша.  Рядом с GT-мешом расположили облако от реконструкции и уронили точки из облака на меш по направлению нормалей меша (ray-casting).

# Libraries

In [ ]:
%load_ext autoreload
%autoreload 2
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars

from pathlib import Path
import pycolmap
import sys

sys.path.append("/workspace/pixel-perfect-sfm/")
sys.path.append("/workspace/pixel-perfect-sfm/Hierarchical-Localization")

from hloc import extract_features, match_features, reconstruction, pairs_from_exhaustive, visualization
from hloc.visualization import plot_images, read_image
from hloc.utils.viz_3d import init_figure, plot_points, plot_reconstruction, plot_camera_colmap
from hloc.utils.read_write_model import  write_next_bytes, Point3D, Image, read_images_text, read_points3D_binary,\
        write_points3D_binary, write_images_binary, read_images_binary, read_cameras_text, read_points3D_text, \
        write_points3D_text, write_images_text

from pixsfm.util.visualize import init_image, plot_points2D
from pixsfm.refine_hloc import PixSfM
from pixsfm import ostream_redirect

import numpy as np
from matplotlib import pyplot as plt

import open3d as o3d
assert o3d.__version__ == '0.15.2', 'The version 0.15.2 is required!'

# redirect the C++ outputs to notebook cells
cpp_out = ostream_redirect(stderr=True, stdout=True)
cpp_out.__enter__()

# Setup

В **object_name** необходимо задать имя объекта, над которым вы хотите провести эксперимент.

**check_for_calibrated_images** - булевая переменная, по которой мы выбираем какие поз камер использовать (менее точные (True) или точные (False))

**delete_previous_output** - если True, то удаляет все предыдущие файлы в папке outputs. Использовать супер осторожно.

**has_cache** - если True, то у Вас уже существует файл с feature maps и он сохранен в папке cache_init. Это файл с feature maps Вы получаете только тогда, когда вы уже сделали featuremetric KA или BA для одного из ваших экспериментов.

**show_visualization** - если True, то показывает визуализацию результата эксперимента (3d pointcloud, задектированные keypoints (features) и final reprojections для какого-то изображения).

In [ ]:
object_name = 'dragon'

check_for_calibrated_images = False
delete_previous_output = False

has_cache = True
show_visualization = False

**images_init** - путь к файлу images.txt с известными позами камер (каждая вторая строка пустая)

**calibrated_images_init** - путь к файлу images.txt c известнами позами камер (но менее точные)

**cameras_init** - путь к файлу cameras.txt

**gt_init** - путь к GT-мешу

**images** - путь к папке с изображениями для реконструкции

**outputs** - путь к папке со всеми результатами

**cache_init** - путь к кэш-файлу, его мы получаем во время того, когда делаем KA или BA. В этот файле хранятся featuremaps после dense feature extraction. В среднем на одну картинку размером 2368х1952 уходит 3 минуты. Этот файл вообще нельзя трогать, поэтому мы копируем его в папку outputs для своего эксперимента и продолжаем работу.

**cache_path** - тот же файл, что cache_init, с которым мы теперь будем работать во время эксперимента.

In [ ]:
root = Path('/workspace')

images_init = root / f'datasets/sk3d/dataset/{object_name}/tis_right/rgb/images.txt'
calibrated_images_init = root / 'datasets/sk3d/dataset/calibration/tis_right/rgb/images.txt' # менее точные

cameras_init = root / 'datasets/sk3d/dataset/calibration/tis_right/rgb/cameras.txt'
gt_init = root / f'datasets/sk3d/dataset/{object_name}/stl/reconstruction/cleaned.ply'

# all images for our object
images = root / f'datasets/sk3d/dataset/{object_name}/tis_right/rgb/undistorted/ambient@best'

# where all result data will be stored
outputs = root / f'pixel-perfect-sfm/outputs/{object_name}_exp3/'

if delete_previous_output:
    !rm -rf $outputs 
    
outputs.mkdir(parents=True, exist_ok=True)    
    
if has_cache:
    cache_init = root / f'pixel-perfect-sfm/outputs/caches/{object_name}/s2dnet_featuremaps_sparse.h5'
    !cp -r $cache_init $outputs
    cache_path = outputs / 's2dnet_featuremaps_sparse.h5'

**sfm_dir** - папка, в которой хранится реконструкция из второго эксперимента. 

**exp3_dir** - папка, в которой будет сохранен результат третьего эксперимента.

In [ ]:
sfm_dir = outputs / "sfm"
sfm_dir.mkdir(parents=True, exist_ok=True)

exp3_dir = outputs / "exp3"
exp3_dir.mkdir(parents=True, exist_ok=True)


if check_for_calibrated_images:
    images_init = calibrated_images_init

    exp3_dir = outputs / "calibrated/exp3"
    exp3_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
!cp -r /workspace/pixel-perfect-sfm/outputs/dragon/ref_exp2/hloc/model/model_txt/*  \
            $sfm_dir

Смотрим саммэри для готовой реконструкции.

In [ ]:
sfm_reconstructiom = pycolmap.Reconstruction(sfm_dir)
print("Reconstruction from second experiment: ", sfm_reconstructiom.summary())

In [ ]:
# get 3d points from our reconstruction
ref_3d_points = {}

# ref_3d_points - dict with keys of points_id and values as points
for point3D_id, point3D in sfm_reconstructiom.points3D.items():
    ref_3d_points.update({point3D_id: list(point3D.xyz)})

In [ ]:
# Code to find distance between points and mesh 'path_to_reference_mesh'
vert = list(ref_3d_points.values())

rec = o3d.io.read_triangle_mesh(str(gt_init))
rec.compute_vertex_normals()

raycasting =  o3d.t.geometry.RaycastingScene()
rec = o3d.t.geometry.TriangleMesh.from_legacy(rec)
raycasting.add_triangles(rec)

# pts has shape [pts_n, 3]
pts = o3d.core.Tensor(vert, dtype=o3d.core.Dtype.Float32)
ans = raycasting.compute_closest_points(pts)
closest_pts = raycasting.compute_closest_points(pts)['points'].numpy()

print('The closest point on the surface is', closest_pts)
print('Number of points ',  len(closest_pts))
dist = np.linalg.norm(closest_pts - pts.numpy(), axis=1)
print('Distance between points and mesh ', dist)

Здесь нужно установить определенный threshold. Мы будем вычислять расстояние между точками и GT-мешом. Если расстояние меньше чем установленный threshold, то эти точки оставляем и записываем в БД, иначе удаляем точки и из меша и в БД. В ланном случае мы выбрали threshold 9 см.

In [ ]:
thresh = 0.09

In [ ]:
dist_1, dist_2 = [], []

pts_all = []
pts_out_range = []
pts_in_range = []

# vert - vertices from second reconstruction
# pts - corresponding closest points for vert

for i in range(len(vert)):
    dist = np.linalg.norm(closest_pts[i] - pts[i]).numpy()
    dist_1.append(dist)
    pts_all.append(pts[i].numpy())
    if 0 < dist < thresh:
        pts_all[i] = closest_pts[i]
        pts_in_range.append(closest_pts[i])
        
        new_dist = np.linalg.norm(pts_all[i] - pts[i]).numpy()
        dist_2.append(new_dist)
    else:
        pts_out_range.append(pts[i].numpy())
        dist_2.append(thresh)
        

plt.plot(dist_1, color='red')
plt.plot(dist_2, color='green')
plt.show()  

Создаем два множества точек: 1) внутри threshold и мы их оставляем   2) за пределами threshold и мы их удаляем. 

In [ ]:
points3d_in = {}
points3d_out = {}

for point3D_id, point3D in sfm_reconstructiom.points3D.items():

    query_point = [list(point3D.xyz)]
    pts = o3d.core.Tensor(query_point, dtype=o3d.core.Dtype.Float32)
    ans = raycasting.compute_closest_points(pts)
    closest_pts = raycasting.compute_closest_points(pts)['points'].numpy()
    dist = np.linalg.norm(closest_pts - pts.numpy(), axis=1)
    
    if 0 < dist < thresh:
        points3d_in.update({point3D_id: closest_pts.ravel()})
    else:
        points3d_out.update({point3D_id: point3D.xyz.astype(np.float32)})

Создаем файл point3D.txt

In [ ]:
# getting ground truth points from second reconstruction

#gt_points = read_points3D_binary( sfm_dir / 'points3D.bin')
gt_points = read_points3D_text(sfm_dir / 'points3D.txt')

points3D = {}

# updating gt_points
for key, value in points3d_in.items():
    if (gt_points[key].xyz != value).all():
        point3D_id = gt_points[key].id
        
        # putting new value here
        xyz = value 
        rgb = gt_points[key].rgb
        error = gt_points[key].error
        image_ids_ = gt_points[key].image_ids
        point2D_idxs = gt_points[key].point2D_idxs
        points3D[point3D_id] = Point3D(id=point3D_id, 
                                       xyz=xyz, 
                                       rgb=rgb,
                                       error=error, 
                                       image_ids=image_ids_,
                                       point2D_idxs=point2D_idxs)


# write_points3D_binary(points3D, exp3_dir / 'points3D.bin')    
write_points3D_text(points3D, exp3_dir / 'points3D.txt')  

Создаем файл images.txt

In [ ]:
#images_dict = read_images_binary( sfm_dir / 'images.bin')
images_dict = read_images_text( sfm_dir / 'images.txt')

updated_images_dict = {}


pts_to_delete = points3d_out.keys()

#updating images dictionary
for k, value in images_dict.items():
    new_point3D_ids = []
    image_id = k
    qvec = value.qvec
    tvec = value.tvec
    camera_id = value.camera_id
    name = value.name
    xys = value.xys
      
    # we need to put -1 for images.txt, because we will not have these points anymore    
    for _id in value.point3D_ids:
        if _id in pts_to_delete:
            new_point3D_ids.append(-1)
        else:
            new_point3D_ids.append(_id)

    updated_images_dict[image_id] = Image(
                id=image_id, 
                qvec=qvec, tvec=tvec,
                camera_id=camera_id, 
                name=name,
                xys=xys, 
                point3D_ids=np.array(new_point3D_ids))

# write_images_binary(updated_images_dict, exp3_dir / 'images.bin')    
write_images_text(updated_images_dict, exp3_dir / 'images.txt')    

Копируем cameras.txt из предыдущего эксперимента.

In [ ]:
!cp -r $sfm_dir/cameras.txt $exp3_dir

**Featuremetric BA (Bundle Adjustment)**

В конфиге conf_BA расписана необходимая информация для того, чтобы сделать BA. В этом конфиге также указано, что вы хотите использовать cache, который находится в пути cache_path.

Здесь описано как настроить конфигурацию для ВА. https://github.com/cvg/pixel-perfect-sfm#detailed-configuration


 Ниже клетка выполняется за 1.5-2 минуты.

In [ ]:
from pixsfm.refine_colmap import PixSfM

conf_BA = {
        "dense_features": {
                "use_cache": True,
        },
        
        "BA": {
            "dense_features": {'use_cache': True}, 
            "apply": True
        }
}

refiner = PixSfM(conf=conf_BA)
reconstruction, ba_data, feature_manager = refiner.refine_reconstruction(
    output_path = exp3_dir / f'thresh_{thresh}',
    input_path = exp3_dir,
    image_dir = images,
    cache_path = cache_path,
)

print(reconstruction.summary())

Перевод модели в формат TXT.

In [ ]:
!mkdir -p $exp3_dir/thresh_$thresh/model_txt/ 

!colmap model_converter \
    --input_path $exp3_dir/thresh_$thresh/ \
    --output_path $exp3_dir/thresh_$thresh/model_txt/ \
    --output_type TXT

# Visualization

In [ ]:
fig3d = init_figure()

args = dict(max_reproj_error=3.0, 
            min_track_length=2, 
            cs=0.01) #camera size
plot_reconstruction(fig3d, reconstruction, 
                    color='rgba(0, 255, 0, 0.5)', 
                    name="refined", **args)
if show_visualization:
    fig3d.show()

In [ ]:
refined = reconstruction

img = refined.images[refined.reg_image_ids()[0]]
cam = refined.cameras[img.camera_id]

fig = init_image(images / img.name)    

plot_points2D(fig, [p2D.xy for p2D in img.points2D if p2D.has_point3D()])
plot_points2D(fig, cam.world_to_image(img.project(refined)), color='rgba(255, 0, 0, 0.5)')

if show_visualization:
    fig.show()